In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
import requests
import pandas as pd
import json
import talib
import numpy as np
import time

import yfinance as yf
import warnings
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')


SECRET_FP = "./secrets.json"
with open(SECRET_FP, 'r') as file:
    secrets = json.load(file)
API_KEY = secrets['eodhd_api_key']

sys.path.append(r"C:\Users\micha\OneDrive\Documents\code\submodule")
from eodhd import  Chain, ChainHistory, Security, Index
import plot
from plot import ChartStyle
cs = ChartStyle(figsize=(20,8),nyears=1, date_fmt="%Y-%m", grid=False)


In [ ]:
sec = Security(ticker="SPY")
sec = Security(ticker=["SPY", 'AAPL', 'MSFT'], start = '2025-01-01')
sec = Security(ticker=["IVV", 'AAPL', 'MSFT'], start = '2025-01-01')

In [ ]:
sec.eoddf
sec.last_close
sec.prices
sec.returns
sec.get_rsi(prices=sec.prices, period=14)
sec.get_bbands(prices=sec.prices, period=20)
sec.get_moving_average(sec.prices, period=50)

In [ ]:
Index(ticker='')